
<p align="center"><img src="https://github.com/Reece323/air-scrape/blob/main/assets/airbnb_logo.png?raw=true" width="300"/></p>
<center> <h1> Analyzing AirBnB Listings </h1> </center>



### This project focuses on data gathered from AirBnB listings
#### Goals:


*   *Manipulating and Exploring the data*
*   *Creating Visualizations*
*   *Use Machine Learning*
*   *Determine which features attracts consumers*
*   *See how to increase revenue as `host`*

In [147]:
# Imports
import pandas as pd
import numpy as np
import json
import urllib.request
from operator import itemgetter
from pprint import pprint
from itertools import chain
import re

# !pip install rich
from rich.console import Console

console = Console()
print = console.print

pd.options.display.max_columns = 4000

import folium
import contextily as cx
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib as mpl
from matplotlib import cm
from colorspacious import cspace_converter

In [148]:
url = 'https://raw.githubusercontent.com/Reece323/air-scrape/main/Data/Bentonville_all.json'
response = urllib.request.urlopen(url)
Bentonville = json.loads(response.read())
Bville = pd.json_normalize(Bentonville)

In [149]:
cols_to_drop_temp = [
    # 'url',
    "calendar",
    'rootAmenitySections',
    'p3SummaryAddress',
    'primaryHost.about',
    'primaryHost.badges',
    'primaryHost.firstName',
    'primaryHost.id',
    'primaryHost.isSuperhost',
    'primaryHost.memberSinceFullStr',
    'primaryHost.name',
    'primaryHost.languages',
    'primaryHost.pictureUrl',
    'primaryHost.responseRateWithoutNa',
    'primaryHost.responseTimeWithoutNa',
    'primaryHost.hasInclusionBadge',
    'primaryHost.pictureLargeUrl',
    'primaryHost.hostIntroTags',
    'primaryHost.hostUrl',
    'primaryHost.listingsCount',
    'primaryHost.totalListingsCount',
    'tierId',
    'hasSpecialOffer',
    'hasWeWorkLocation',
    'hasWeWorkLocation',
    'additionalHosts',
    'hometourRooms',
    'hometourSections',
    'descriptionLocale',
    'initialDescriptionAuthorType',
    'localizedCheckInTimeWindow',
    'localizedCheckOutTime',
    'hometourRooms',
    'countryCode',
    'hasHostGuidebook',
    'hasLocalAttractions',
    'neighborhoodCommunityTags',
    'state',
    'paidGrowthRemarketingListingIds',
    'hasCommercialHostInfo',
    'hostSignatureFont.url',
    'nearbyAirportDistanceDescriptions',
    'propertyTypeInCity',
    'renderTierId',
    'isHotel',
    'isNewListing',
    'showReviewTag',
    'isRepresentativeInventory',
    'localizedCity',
    'highlights',
    'highlightsImpressionId',
    'pointOfInterests',
    'hostGuidebook.guidebookUrl',
    'hostGuidebook.localizedNameForHomesPdp',
    'hostGuidebook.title',
    'hostGuidebook.id',
    'pageViewType',
    'previewTags',
    'seeAllHometourSections',
    'enableHighlightsVoting',
    'heroModule.categorizedPhotos',
    'sortedReviews',
    'documentDisplayPictures',
    'sections',
    'p3ImpressionId',
    'paidGrowthRemarketingListingIdsStr',
    'isBusinessTravelReady',
    'address',
    'hasHouseRules',
    'reviewsOrder',
    'hostQuote',
    'localizedListingExpectations',
    'pricing.rate.amount_formatted',
    'pricing.rate.currency',
    'pricing.rate.is_micros_accuracy',
    'pricing.rate_with_service_fee.amount_formatted',
    'pricing.rate_with_service_fee.currency',
    'pricing.rate_with_service_fee.is_micros_accuracy',
    'reviewDetailsInterface.reviewSummary',
    'reviewsModule.appreciationTags',
    'pricing.rate_with_service_fee.amount',
    'pricing.rate_type',
    'sectionedDescription.authorType',
    'sectionedDescription.locale',
    'sectionedDescription.localizedLanguageName',
    'sectionedDescription.notes',
    'sectionedDescription.space',
    'guestControls.allowsNonChinaUsers',
    'guestControls.structuredHouseRulesWithTips',
    'sectionedDescription.neighborhoodOverview',
    'sectionedDescription.access',
    'sectionedDescription.description',
    'sectionedDescription.interaction',
    'sectionedDescription.name',
    'sectionedDescription.summary',
    'sectionedDescription.transit',
    # 'name',
    'listingExpectations',
    'sectionedDescription.houseRules',
    'additionalHouseRules',
    'idStr'
]

Bville = Bville.drop(cols_to_drop_temp, axis=1)

In [150]:
df = Bville.reindex(columns=[
    'url',
    'name',
    'city',
    'country',
    'id',
    'location.lat',
    'location.lng',
    'numberOfGuests',
    'listingRooms',
    'roomType',
    'roomTypeCategory',
    'bedroomLabel',
    'bedLabel',
    'bathroomLabel',
    'listingAmenities',
    'seeAllAmenitySections',
    'categorizedPreviewAmenities',
    'stars',
    'reviewsModule.localizedOverallRating',
    'reviewDetailsInterface.reviewCount',
    'reviews',
    'isHostedBySuperhost',
    'maxNights',
    'minNights',
    'priceDetails',
    'pricing.rate.amount',
    'guestControls.allowsChildren',
    'guestControls.allowsEvents',
    'guestControls.allowsInfants',
    'guestControls.allowsPets',
    'guestControls.allowsSmoking',
    'guestControls.personCapacity',
    'photos',
    'guestControls.structuredHouseRules',
]
)


In [116]:
df.columns = [re.sub("[ ,\$]", "_", re.sub("[.]", "", c)) for c in df.columns]
print(df.columns)
df

Index(['city', 'country', 'id', 'locationlat', 'locationlng', 'numberOfGuests',
       'listingRooms', 'roomType', 'roomTypeCategory', 'bedroomLabel',
       'bedLabel', 'bathroomLabel', 'listingAmenities',
       'seeAllAmenitySections', 'categorizedPreviewAmenities', 'stars',
       'reviewsModulelocalizedOverallRating',
       'reviewDetailsInterfacereviewCount', 'reviews', 'isHostedBySuperhost',
       'maxNights', 'minNights', 'priceDetails', 'pricingrateamount',
       'guestControlsallowsChildren', 'guestControlsallowsEvents',
       'guestControlsallowsInfants', 'guestControlsallowsPets',
       'guestControlsallowsSmoking', 'guestControlspersonCapacity',
       'guestControlsstructuredHouseRules'],
      dtype='object')

,city,country,id,locationlat,locationlng,numberOfGuests,listingRooms,roomType,roomTypeCategory,bedroomLabel,bedLabel,bathroomLabel,listingAmenities,seeAllAmenitySections,categorizedPreviewAmenities,stars,reviewsModulelocalizedOverallRating,reviewDetailsInterfacereviewCount,reviews,isHostedBySuperhost,maxNights,minNights,priceDetails,pricingrateamount,guestControlsallowsChildren,guestControlsallowsEvents,guestControlsallowsInfants,guestControlsallowsPets,guestControlsallowsSmoking,guestControlspersonCapacity,guestControlsstructuredHouseRules
0,Bentonville,United States,32721035,36.40187,-94.22518,4,"[{'beds': [{'id': '37190702/queen_bed', 'quant...",Entire guest suite,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.96,366,"[{'author': {'firstName': 'Zach', 'hasProfileP...",True,29,2,"[{'label': 'Cleaning Fee', 'value': '$20', 'at...",49,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec..."
1,Bentonville,United States,14023239,36.37226,-94.21269,4,"[{'beds': [{'id': '8053552/sofa_bed', 'quantit...",Entire guesthouse,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5,4.97,329,"[{'author': {'firstName': 'Savannah', 'hasProf...",True,14,1,"[{'label': 'Cleaning Fee', 'value': '$50', 'at...",92,True,False,True,False,False,4,"[No pets, No smoking, parties, or events, Chec..."
2,Bentonville,United States,38783656,36.37763,-94.18872,2,"[{'beds': [{'id': '65625913/king_bed', 'quanti...",Private room in residential home,private_room,1 bedroom,1 bed,1 private bath,"[{'description': '', 'id': 8, 'isPresent': Fal...","[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Wifi, ...",5,4.92,239,"[{'author': {'firstName': 'Rachel', 'hasProfil...",True,28,2,"[{'label': 'Cleaning Fee', 'value': '$10', 'at...",56,True,False,False,False,False,2,[Not safe or suitable for infants (under 2 yea...


In [151]:
df.columns = [re.sub("[ ,\$]", "_", re.sub("[.]", "", c)) for c in df.columns]

df['geometry'] = [(xy) for xy in zip(df.locationlat, df.locationlng)]

df

,url,name,city,country,id,locationlat,locationlng,numberOfGuests,listingRooms,roomType,roomTypeCategory,bedroomLabel,bedLabel,bathroomLabel,listingAmenities,seeAllAmenitySections,categorizedPreviewAmenities,stars,reviewsModulelocalizedOverallRating,reviewDetailsInterfacereviewCount,reviews,isHostedBySuperhost,maxNights,minNights,priceDetails,pricingrateamount,guestControlsallowsChildren,guestControlsallowsEvents,guestControlsallowsInfants,guestControlsallowsPets,guestControlsallowsSmoking,guestControlspersonCapacity,photos,guestControlsstructuredHouseRules,geometry
0,https://www.airbnb.com/rooms/32721035,Cozy & Relaxing Suite: Pool/Movie Screen/Trails,Bentonville,United States,32721035,36.40187,-94.22518,4,"[{'beds': [{'id': '37190702/queen_bed', 'quant...",Entire guest suite,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.96,366,"[{'author': {'firstName': 'Zach', 'hasProfileP...",True,29,2,"[{'label': 'Cleaning Fee', 'value': '$20', 'at...",49,True,False,True,False,False,4,[{'caption': 'Relax poolside after a day of ex...,"[No pets, No smoking, parties, or events, Chec...","(36.40187, -94.22518)"
1,https://www.airbnb.com/rooms/14023239,"Cozy Garden Retreat in Downtown - Patio, Bike ...",Bentonville,United States,14023239,36.37226,-94.21269,4,"[{'beds': [{'id': '8053552/sofa_bed', 'quantit...",Entire guesthouse,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.97,329,"[{'author': {'firstName': 'Savannah', 'hasProf...",True,14,1,"[{'label': 'Cleaning Fee', 'value': '$50', 'at...",92,True,False,True,False,False,4,"[{'caption': 'A cozy, comfortable, light-fille...","[No pets, No smoking, parties, or events, Chec...","(36.37226, -94.21269)"
2,https://www.airbnb.com/rooms/38783656,Memorial Park Hideaway - near Crystal Bridges,Bentonville,United States,38783656,36.37763,-94.18872,2,"[{'beds': [{'id': '65625913/king_bed', 'quanti...",Private room in residential home,private_room,1 bedroom,1 bed,1 private bath,"[{'description': '', 'id': 8, 'isPresent': Fal...","[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Wifi, ...",5.0,4.92,239,"[{'author': {'firstName': 'Rachel', 'hasProfil...",True,28,2,"[{'label': 'Cleaning Fee', 'value': '$10', 'at...",56,True,False,False,False,False,2,"[{'caption': '', 'id': 1106743335, 'large': 'h...",[Not safe or suitable for infants (under 2 yea...,"(36.37763, -94.18872)"
3,https://www.airbnb.com/rooms/45565472,Studio w/Deck (Private),Bentonville,United States,45565472,36.37142,-94.22504,4,"[{'beds': [{'id': '107302929/king_bed', 'quant...",Entire residential home,entire_home,Studio,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.97,67,"[{'author': {'firstName': 'Jessica', 'hasProfi...",True,1125,2,"[{'label': 'Cleaning Fee', 'value': '$25', 'at...",89,True,False,True,False,False,4,"[{'caption': 'Sofa bed, king bed', 'id': 10926...","[No pets, No smoking, parties, or events, Chec...","(36.37142, -94.22504)"
4,https://www.airbnb.com/rooms/45831904,Spacious private bedroom located close to down...,Bentonville,United States,45831904,36.36268,-94.18553,2,"[{'beds': [{'id': '110242780/couch', 'quantity...",Private room in residential home,private_room,1 bedroom,1 bed,1 shared bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",4.5,4.69,16,"[{'author': {'firstName': 'Jiaheng', 'hasProfi...",True,365,2,"[{'label': 'Extra people', 'value': 'No Charge...",30,True,True,True,True,False,2,"[{'caption': '', 'id': 1080898520, 'large': 'h...","[No smoking, Che

In [152]:
photoss = df['photos'][0]
pics = [d['large'] for d in photoss if 'large' in d]

In [153]:
coords = [36.3729, -94.2086]
lat = list(df['locationlat'])
lon = list(df['locationlng'])
name = list(df['name'])
url = list(df['url'])
rate = list(df['reviewsModulelocalizedOverallRating'])
price = list(df['pricingrateamount'])
pics

['https://a0.muscache.com/im/pictures/43831c7d-21e3-4288-8e98-8a606e594e83.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/e6776b23-6456-45fb-a4d1-ac7ad762dc94.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/06bd166b-eaa9-441f-a514-2840a3061325.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/ea6876db-b3b7-41ed-a429-d6117feff919.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/miso/Hosting-32721035/original/0eabdcba-446b-4d4d-b6c8-17fb66237c7a.jpeg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/miso/Hosting-32721035/original/d234888a-785b-4a69-b93e-dd0a1b316e6a.jpeg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/miso/Hosting-32721035/original/abc4337f-1df8-4fe0-ba31-208a1cd28acf.jpeg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/miso/Hosting-32721035/original/e1ca6e5e-3ccb-4a41-b0dc-adb8f8a96f33.jpeg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/bc83ab72-de06-4d24-8149-08c41d5935d7.jpg?aki_polic

In [154]:
map=folium.Map(location=coords,zoom_start=11)
fg=folium.FeatureGroup(name="my map")

In [155]:
for lt, ln, nm, site, rater, prc in zip(lat, lon, name, url, rate, price):
  fg.add_child(folium.Marker(location=[lt,ln],
        popup= '<b> <blockquote> <center>' + nm + "</center> </b> </blockquote> <br> <center> <b>$"+ prc +" /night <h4> Rated: </h4> </b> </center> <center> <h4><p>&#127814; "+str(rater)+"</h4> </center> <center> <img width='250'src='https://github.com/Reece323/air-scrape/blob/main/assets/airbnb_logo.png?raw=true' /></center> <a href="+ site + "> <center><b><h5>Go to Listing</h5></b></center></a>",
        icon=folium.Icon(color='green')))

map.add_child(fg)